In [97]:
SEASON = '2021-22'

In [98]:
import pandas as pd
import myconstants as c
from scipy.stats import zscore

NUM_GAMES = c.NUM_GAMES_2020_21
MIN_GAMES = NUM_GAMES * 0.60

p_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_PerGame.csv')
t_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Team_Standings.csv')

# join player per game and advanced stats
p_advanced_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_Advanced.csv')
p_advanced_df = p_advanced_df.drop(columns=['Player', 'Tms', 'Pos', 'Age', 'G', 'MP'])
p_df = p_df.merge(p_advanced_df, on='PlayerID')

# populate team df with team codes
t_df['TeamCode'] = t_df['Team'].apply(lambda x: c.TEAM_CODE[x])

# average team wins per player
def average_wins(tms):
    tms_list = tms.split(',')
    tms_wins = []
    for t in tms_list:
        tms_wins.append(t_df.loc[t_df['TeamCode'] == t]['OverallW'].values[0])
    return sum(tms_wins) / len(tms_wins)
p_df['TmsWins'] = p_df['Tms'].apply(average_wins)

# players who haven't played enough games don't qualify
p_df = p_df.loc[p_df['G'] >= MIN_GAMES]

# ranking algorithm
p_df['PlayerScore'] = 1.5*zscore(p_df['PTS'] + p_df['TRB'] + p_df['AST'] + p_df['STL'] + p_df['BLK']) + zscore(p_df['WS']) + zscore(p_df['VORP']) + zscore(p_df['BPM']) + zscore(p_df['TmsWins'])
p_df = p_df.sort_values('PlayerScore', ascending=False)

# Move [PlayerScore] to the right of [Pos], drop PlayerID, index rows starting from 1, stylistic choice
col = p_df.pop('PlayerScore')
p_df.insert(p_df.columns.get_loc('Pos') + 1, col.name, col)
p_df = p_df.drop('PlayerID', axis=1)
p_df = p_df.reset_index(drop=True)
p_df.index = p_df.index + 1

p_df.head(50)

,Player,Tms,Pos,PlayerScore,Age,G,GS,MP,FG,FGA,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,TmsWins
1,Nikola Jokić,DEN,C,22.060036,26,74,74,33.5,10.3,17.7,...,31.9,10.8,4.5,15.2,0.296,9.2,4.5,13.7,9.8,48.0
2,Giannis Antetokounmpo,MIL,PF,18.531342,27,67,67,32.9,10.3,18.6,...,34.9,9.2,3.7,12.9,0.281,7.6,3.5,11.2,7.4,51.0
3,Joel Embiid,PHI,C,16.665660,27,68,68,33.8,9.8,19.6,...,37.2,7.9,4.1,12.0,0.252,7.2,2.0,9.2,6.5,51.0
4,Luka Dončić,DAL,PG,14.048283,22,65,65,35.4,9.9,21.6,...,37.4,3.8,3.8,7.6,0.159,6.4,1.8,8.2,5.9,52.0
5,Kevin Durant,BRK,PF,12.051955,33,55,55,37.2,10.5,20.3,...,31.2,6.4,2.0,8.4,0.198,6.4,0.7,7.2,4.8,44.0
6,Jayson Tatum,BOS,SF,11.556941,23,76,76,35.9,9.3,20.6,...,32.1,5.0,4.6,9.6,0.169,4.3,0.6,4.9,4.8,51.0
7,Trae Young,ATL,PG,11.402080,23,76,76,34.9,9.4,20.3,...,34.4,9.0,1.0,10.0,0.181,7.1,-2.0,5.2,4.8,43.0
8,LeBron James,LAL,SF,11.358011,37,56,56,37.2,11.4,21.8,...,32.3,5.2,2.3,7.5,0.172,6.9,0.8,7.7,5.1,33.0
9,Karl-Anthony Towns,MIN,C,10.990553,26,74,74,33.5,8.7,16.4,...,27.8,7.1,3.2,10.3,0.199,4.5,0.5,5.0,4.4,46.0
10,Chris Paul,PHO,PG,10.845223,36,65,65,32.9,5.6,11.3,...,19.7,5.8,3.6,9.4,0.210,3.1,2.3,5.4,4.0,64.0
